In [11]:
import time
import numpy as np
import qiskit
import dimod
from qiskit import QuantumCircuit, transpile
from qiskit.visualization import *
from qiskit_optimization.problems import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms import NumPyMinimumEigensolver
from qiskit_algorithms import QAOA
from qiskit_aer import Aer
from qiskit_algorithms.optimizers import COBYLA
from qiskit_algorithms.optimizers import SPSA
from qiskit.primitives import BackendSampler
from qiskit.primitives import Sampler
from qiskit_optimization import QuadraticProgram

In [3]:
Diccionario={(1, 1): -9.626557575259653e-05, (2, 2): -1.726328109796782e-05, (3, 3): -0.00010051278615308412, (4, 4): -0.0002036557915305881, (1, 2): 1.2323167015803069e-05, (1, 3): 5.824880612740829e-06, (1, 4): 3.928456524946224e-05, (2, 1): 1.2323167015803069e-05, (2, 3): 5.223848725612105e-05, (2, 4): 2.58749949503751e-05, (3, 1): 5.824880612740829e-06, (3, 2): 5.223848725612105e-05, (3, 4): 2.6338325684943215e-05, (4, 1): 3.928456524946224e-05, (4, 2): 2.58749949503751e-05, (4, 3): 2.6338325684943215e-05, (5, 5): -3.0972446e-06, (1, 5): 5.070411657204232e-07, (5, 1): 5.070411657204232e-07, (2, 5): 1.100178664836804e-06, (5, 2): 1.100178664836804e-06, (3, 5): 5.139288721534019e-07, (5, 3): 5.139288721534019e-07, (4, 5): 7.346003770836163e-07, (5, 4): 7.346003770836163e-07}

In [9]:
def QAOA_Solution2(Diccionario):

    #Transforma el resultado en uno con variables primadas
    Indice_Maximo = max(max(key) for key in Diccionario)
    Diccionario_Primado={}
    for i in range(0,Indice_Maximo+1):
        for j in range(0,Indice_Maximo+1):
            if (i,j) in Diccionario:
                Diccionario_Primado[(f'{i}',f'{j}')]=Diccionario[(i,j)]
    print(Diccionario_Primado)
    

    #Genera el problema
    qp = QuadraticProgram()
    for i in range(1,Indice_Maximo+1):
        if (i,i) in Diccionario:
            qp.binary_var(f'{i}')

    qp.minimize(quadratic = Diccionario_Primado)
    print(qp.export_as_lp_string())


    sim = Aer.get_backend('aer_simulator_statevector')
    sampler = BackendSampler(sim)
    spsa = SPSA(maxiter=250)
    qaoa = QAOA(sampler=sampler, optimizer=spsa, reps=3)
    qaoa_optimizer = MinimumEigenOptimizer(qaoa)
    result2 = qaoa_optimizer.solve(qp)
    res2 = np.array(result2.x)
    print('QAOA')
    print(result2)


    return res2

In [19]:
QAOA_Solution2(Diccionario)

{('1', '1'): -9.626557575259653e-05, ('1', '2'): 1.2323167015803069e-05, ('1', '3'): 5.824880612740829e-06, ('1', '4'): 3.928456524946224e-05, ('1', '5'): 5.070411657204232e-07, ('2', '1'): 1.2323167015803069e-05, ('2', '2'): -1.726328109796782e-05, ('2', '3'): 5.223848725612105e-05, ('2', '4'): 2.58749949503751e-05, ('2', '5'): 1.100178664836804e-06, ('3', '1'): 5.824880612740829e-06, ('3', '2'): 5.223848725612105e-05, ('3', '3'): -0.00010051278615308412, ('3', '4'): 2.6338325684943215e-05, ('3', '5'): 5.139288721534019e-07, ('4', '1'): 3.928456524946224e-05, ('4', '2'): 2.58749949503751e-05, ('4', '3'): 2.6338325684943215e-05, ('4', '4'): -0.0002036557915305881, ('4', '5'): 7.346003770836163e-07, ('5', '1'): 5.070411657204232e-07, ('5', '2'): 1.100178664836804e-06, ('5', '3'): 5.139288721534019e-07, ('5', '4'): 7.346003770836163e-07, ('5', '5'): -3.0972446e-06}
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: [ - 0.000192531152 x1^2

array([1., 0., 1., 1., 0.])

In [18]:
def Quantum_annealing_simulator(Diccionario):
    J=Diccionario
    h={}
    problem = dimod.BinaryQuadraticModel(h, J, 0.0, dimod.BINARY)

    
    solver2 = dimod.ExactSolver()
    response = solver2.sample(problem)


    print(response)
    
    return response

In [17]:
Quantum_annealing_simulator(Diccionario)

    1  2  3  4  5    energy num_oc.
9   1  0  1  1  0 -0.000258       1
22  1  0  1  1  1 -0.000257       1
23  0  0  1  1  1 -0.000252       1
8   0  0  1  1  0 -0.000251       1
17  1  0  0  1  1 -0.000222       1
14  1  0  0  1  0 -0.000221       1
16  0  0  0  1  1 -0.000205       1
15  0  0  0  1  0 -0.000204       1
25  1  0  1  0  1 -0.000186       1
6   1  0  1  0  0 -0.000185       1
12  0  1  0  1  0 -0.000169       1
19  0  1  0  1  1 -0.000169       1
13  1  1  0  1  0 -0.000162       1
18  1  1  0  1  1 -0.000161       1
11  0  1  1  1  0 -0.000113       1
20  0  1  1  1  1 -0.000111       1
24  0  0  1  0  1 -0.000103       1
7   0  0  1  0  0 -0.000101       1
30  1  0  0  0  1 -0.000098       1
1   1  0  0  0  0 -0.000096       1
10  1  1  1  1  0 -0.000094       1
21  1  1  1  1  1 -0.000091       1
2   1  1  0  0  0 -0.000089       1
29  1  1  0  0  1 -0.000089       1
5   1  1  1  0  0 -0.000073       1
26  1  1  1  0  1 -0.000072       1
28  0  1  0  0  1 -0.000018 

SampleSet(rec.array([([0, 0, 0, 0, 0],  0.00000000e+00, 1),
           ([1, 0, 0, 0, 0], -9.62655758e-05, 1),
           ([1, 1, 0, 0, 0], -8.88825228e-05, 1),
           ([0, 1, 0, 0, 0], -1.72632811e-05, 1),
           ([0, 1, 1, 0, 0], -1.32990927e-05, 1),
           ([1, 1, 1, 0, 0], -7.32685732e-05, 1),
           ([1, 0, 1, 0, 0], -1.85128601e-04, 1),
           ([0, 0, 1, 0, 0], -1.00512786e-04, 1),
           ([0, 0, 1, 1, 0], -2.51491926e-04, 1),
           ([1, 0, 1, 1, 0], -2.57538610e-04, 1),
           ([1, 1, 1, 1, 0], -9.39285930e-05, 1),
           ([0, 1, 1, 1, 0], -1.12528243e-04, 1),
           ([0, 1, 0, 1, 0], -1.69169083e-04, 1),
           ([1, 1, 0, 1, 0], -1.62219194e-04, 1),
           ([1, 0, 0, 1, 0], -2.21352237e-04, 1),
           ([0, 0, 0, 1, 0], -2.03655792e-04, 1),
           ([0, 0, 0, 1, 1], -2.05283835e-04, 1),
           ([1, 0, 0, 1, 1], -2.21966198e-04, 1),
           ([1, 1, 0, 1, 1], -1.60632798e-04, 1),
           ([0, 1, 0, 1, 1], -1.68596769